In [ ]:
# Imports
import numpy as np
import random
import math
# import networkx as nx
# from scipy.spatial.distance import pdist, squareform
# import matplotlib.pyplot as plt

In [ ]:
# Function and CLass definitions

class Poster(ink_total, inks): # stores details of a poster
    def __init__(self, ink_total, inks):
        
        #ink_total referring to total number of inks used for this poster
        self.ink_total = ink_total
        # inks referring a list of inks used for this poster [ints]
        self.inks = inks
        
        #TODO: 
    
class Print_instance(inks, poster_count = 1): # i.e. unit of print time (can consist of multiple posters)
    def __init__(self, inks, poster_count):
        # inks referring to number a list of inks used in the print instance
        self.final_inks = inks
        # posterCount referring to the number of posters to be printed in this print instance.
        self.poster_count = poster_count
        
        #TODO: D
    
def get_order_list():
    """Getting and returning order_list from input"""
    # each poster in list should be of object type Poster
    # TODO: get orders list input
    
def elitism_selection(sorted_population, max_pop_size, survivor_count):
    """
    "Keep best half" selection approach
    """
    
    elitism_population = []   

    for i in range(max_pop_size - survivor_count):
        elitism_population.append(sorted_population[i])
            
    return elitism_population

def parent_selection(sorted_population, max_pop_size):
    # TODO: Consider survivor selection options (µ, lambda) or (µ + lambda)

# TODO: consider different crossover techniques(?), e.g. N-point crossover
def single_point_crossover(parent1, parent2):
    crossover_index = random.randint(0, len(parent1))
    
    # 1st parent
    p1_left = parent1[0:crossover_index]
    p1_right = parent1[crossover_index:(len(parent1) - 1)]
    # 2nd parent
    p2_left = parent2[0:crossover_index]
    p2_right = parent2[crossover_index:(len(parent1) - 1)]
    
    ##############################################################################################################
    while i < len(parent1):
        if 
    child1
    child2
    
    # TODO: return 2 children (crossed over, likely?)
    
def mutate(offspring):
    # TODO: return offspring (likely 8-queen problem approach, likely?)
    
def tetrisify(candidate):
    """[summary]
    :param candidate:
    :returns: List of Print_instance objects
    """
    
    tetrified_candidate = []
    # TODO: candidate begins as Print_instance list
    # Converting Poster list to Print_instance list
    for poster in candidate:
        tetrified_candidate.append(Print_instance(poster.inks))
    
    # Checking for duplication of inks 
    # len(candidate)- 1 so that last poster does not check
    for i in range(len(tetrified_candidate) - 1):
        can_compress = True
        for ink in tetrified_candidate[i].final_inks:
            # For ink in poster...
            if ink in tetrified_candidate[i + 1].final_inks:
                # If ink in next poster's list of inks
                can_compress = False
                break
        
        # Creating combined or individual Print_instance objects to append
        # to tetrisified_candidate list.
        if can_compress:
            # If no duplicated inks...
            combined_inks = tetrified_candidate[i].final_inks + tetrified_candidate[i + 1].final_inks
            #print_instance = Print_instance(combined_inks, 2)
            tetrified_candidate[i].final_inks = combined_inks
            tetrified_candidate[i].poster_count += 1
            
            # Deleting 2nd print_instance, as it is combined
            del tetrified_candidate[i + 1]
            i -= 1
    
    return tetrified_candidate

def fitness(candidate, T, N, success_income = 10, fail_cost = 5):
    """
    Calculating a fitness score of a candidate solution, given the T constraint
    """
    tetrisified_candidate = tetrisify(candidate)
    
    income = 0
    loss = 0
    
    # For each Print_instance unit of time, until time constraint T... 
    for i in range(T):
        poster_count = tetrisified_candidate[i].poster_count
        
        if tetrisified_candidate[i].final_inks < N:
            income = poster_count * success_income
        else:
            loss = poster_count * fail_cost
            
    return income - loss
    
    
def sort_by_fitness(population):
    # TODO: Sort function using fitness function
    
    return sorted_population

def get_next_generation(sorted_population, max_pop_size, survivor_count, mutation_rate):    
    # TODO: not rely on this
    # Using elitism_selection (best half) approach
    
    # TODO: implement other selection methods
    survivor_population = elitism_selection(sorted_population, max_pop_size, survivor_count)
    # for:
    single_point_crossover()
        # TODO: Crossover:
            # TODO: Consider even / odd of survivor population
        # TODO: Mutation:
        
    
    # TODO: parents = parent_selection(sorted_population, max_pop_size)
    max_pop_size = len(sorted_population)
    
    
def genetic_algorithm(orders, T, N, max_pop_size = 20, max_generations = 100, survivor_count = 10, mutation_rate = 0.1):
    """
    [DESC ALAJKSDLASKJDASLKDJASLDKJASLDKAJSDLAKSJDLAKSJDALSKDJASLDKJASLKDJASD]
    :param orders: 
    :param T: Time constraint
    :param N: Total Ink count (2 <= N <= 32) -- Consider odd/even calculations
    :param max_pop_size:
    :param max_generations:
    """
    
    # Defining initial population with random permutations
    population = [] 
    for i in range (max_pop_size):
        # TODO: check this permutation implementation
        printOrders = np.random.permutation(orders)
        population.append(printOrders)
        
    # Performance-tracking Variables
    optimal_solution_found = None
    # TODO: convert solution to solution print order(below)
    optimal_print_order = []
    optimal_fitness_score = -np.inf
    generation_bests = []
    
    # Generation loop
    for i in range(max_generations):
        # Sorting population by fitness
        sorted_population = sort_by_fitness(population)
        
        # TODO: consider population average for analysis (graph representation)
        # Comparing population optimal 
        population_best = fitness(sorted_population[0], T, N) #TODO: total print costs
        if population_best > optimal_fitness_score: # Maximisation 
            optimal_fitness_score = population_best
            optimal_solution_found = sorted_population[0]
        generation_bests.append(population_best)
        
        # TODO: consider here an option for selecting selection method(s), for critical evaluation
        population = get_next_generation(sorted_population, max_pop_size, survivor_count, mutation_rate)        
        
    return optimal_solution_found, optimal_fitness_score
    
    

    

In [ ]:
# Main Function
orders = get_order_list()

genetic_algorithm(orders, params )

In [ ]:
# Plotting Results